![Microsoft-Studios](images/microsoft-studios-banner.png)

# **Microsoft Movie Studio EDA**

---

### Author: [Femi Kamau](https://github.com/ctrl-Karugu)

## Overview
(A one-paragraph overview of the project, including the business problem, data, methods, results and recommendations.)

## Business Problem
* Microsoft has decided to enter the original video content scene by creating a new movie studio. However, they don't know anything about creating movies. In order to solve this problem, the analysis shall be centered around answering the following question:

   > ##### What types of films are currently performing the best at the box office?

* According to this [Holywood Reporter](https://www.hollywoodreporter.com/movies/movie-news/what-is-profitable-movie-ever-1269879/) article, we see that the metric used to determine performance of a movie at the box office may vary. Therefore, it is important to clarify that within this analysis, **the metric that shall be used to determine the success of a movie will be the return on investment (ROI)**. ROI is an important performance measure used by businesses to evaluate the profitability of an investment or compare the efficiency of a number of different investments.

* In order to further understand the types of movies that are currently performing the best at the box office, this analysis will look into the impact following features have on the ROI:

  * Genre
   
  * Runtime (Length)

  * Director(s) & Writer(s)

  * Cast

  * Gross Revenue

  * Rating

  * Production Cost

## Data Understanding
* Within this phase analysis, we will be examining datasets obtained from three renowned film database websites: [The Numbers](https://www.the-numbers.com/), [IMDB](https://www.imdb.com/), and [Rotten Tomatoes](https://www.rottentomatoes.com/). The goal is to explore the general structure of the datasets that we are dealing with, as well as to verify the quality of the data itself.

##### Load Libraries

In [1]:
# NumPy for numerical analysis
import numpy as np

# Pandas for data analysis
import pandas as pd

# Matplotlib.pyplot, Seaborn, and Plotly.express for data visualization
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# Zipfile used to access the data from a zip file
import zipfile

# Sqlite3 for database management
import sqlite3

##### Load Data 

In [2]:
# Function is created to load the data
def load_data(path, data_format):
    
    # if statement used to differentiate data format. Checks if data is in 'csv' format
    if data_format.lower() == 'csv':
        data = pd.read_csv(path, compression='gzip') # 'compression' parameter used since the file is compressed
    
    # elif statement used to check if data is in 'tsv' format  
    elif data_format.lower() == 'tsv':
        data = pd.read_csv(path, compression='gzip', sep='\t', encoding='latin1') # 'sep' parameter used since the file is tab-delimited (tab separated values)
        
    # elif statement used to check if data is in 'db' format
    elif data_format.lower() == 'db':
        conn = sqlite3.connect(path) # returns a Connection object that we will use to interact with the database
        return conn
    # else statement returns an error if the data format is not recognized
    else:
        raise ValueError('Data format not recognized')
    
    return data # returns the dataframe under the variable name 'data'

##### Visualize Missing Data

In [42]:
# Function is created to identify and visualize the missing data
def missing_values(data):
    miss_val = data.isnull().sum().sort_values(ascending = False)
    
    # percentages of missing values per column
    percentages = (data.isnull().sum() / len(data)).sort_values(ascending = False)
    
    # create a dataframe with the missing value percentages per column
    missing_values_df = pd.DataFrame([miss_val.index, percentages * 100], index=['Columns', 'Percentage (%)']).transpose()
    
    # create a bar plot of the missing value
    fig = px.bar(missing_values_df, x='Columns', y='Percentage (%)', title='Percentage of Missing Values per Column', range_y=[0, 100])
    fig.update_layout(title_x=0.5)


    
    return fig

##### Dataset 1: Rotten Tomatoes - Movie Information

In [4]:
# Load the movie information as a pandas DataFrame and assign it to the variable 'movie_info_df'
movie_info_df = load_data('data/rt.movie_info.tsv.gz', 'tsv')

# Preview the first few rows of the DataFrame
movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [5]:
# General description of the movie_info_df data
movie_info_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1560 non-null   int64 
 1   synopsis      1498 non-null   object
 2   rating        1557 non-null   object
 3   genre         1552 non-null   object
 4   director      1361 non-null   object
 5   writer        1111 non-null   object
 6   theater_date  1201 non-null   object
 7   dvd_date      1201 non-null   object
 8   currency      340 non-null    object
 9   box_office    340 non-null    object
 10  runtime       1530 non-null   object
 11  studio        494 non-null    object
dtypes: int64(1), object(11)
memory usage: 146.4+ KB


In [6]:
# Graphical representation of the missing values in the DataFrame
missing_values(movie_info_df)

Analysis of the dataset obtained from [Rotten Tomatoes](https://www.rottentomatoes.com/)

##### Dataset 2: Rotten Tomatoes - Reviews

In [7]:
# Load the movie reviews as a pandas DataFrame and assign it to the variable 'movie_reviews_df'
movie_reviews_df = load_data('data/rt.reviews.tsv.gz', 'tsv') 

# Preview the first few rows of the DataFrame
movie_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


In [8]:
# General description of the 'movie_reviews_df' data
movie_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54432 entries, 0 to 54431
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          54432 non-null  int64 
 1   review      48869 non-null  object
 2   rating      40915 non-null  object
 3   fresh       54432 non-null  object
 4   critic      51710 non-null  object
 5   top_critic  54432 non-null  int64 
 6   publisher   54123 non-null  object
 7   date        54432 non-null  object
dtypes: int64(2), object(6)
memory usage: 3.3+ MB


In [9]:
# Visualize the missing values in the 'movie_reviews_df' DataFrame
missing_values(movie_reviews_df)

Analysis of the dataset obtained from [Rotten Tomatoes](https://www.rottentomatoes.com/)

##### Dataset 3: The Numbers - Movie Budgets

In [10]:
# Load the movie budgets as a pandas DataFrame and assign it to the variable 'movie_budgets_df'
movie_budgets_df = load_data('data/tn.movie_budgets.csv.gz', 'csv')

# Preview the first few rows of the 'movie_budgets_df' DataFrame
movie_budgets_df.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [11]:
# General description of the 'movie_budgets_df' data
movie_budgets_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5782 entries, 0 to 5781
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 5782 non-null   int64 
 1   release_date       5782 non-null   object
 2   movie              5782 non-null   object
 3   production_budget  5782 non-null   object
 4   domestic_gross     5782 non-null   object
 5   worldwide_gross    5782 non-null   object
dtypes: int64(1), object(5)
memory usage: 271.2+ KB


(Analysis of the dataset obtained from [The Numbers](https://www.the-numbers.com/))

##### Dataset 4: IMDB
* The ERD of the IMDB dataset is shown below:

![IMDB Data ERD](images/imdb_data_erd.jpeg)


In [13]:
# Load the imdb data into a dataframe which returns a Connection object
conn = load_data('data/im.db/im.db', 'db')

# View the tables within the idmb database as a dataframe
imdb_tables = pd.read_sql("""SELECT name FROM sqlite_master WHERE type = 'table';""", conn)
imdb_tables

,name
0,movie_basics
1,directors
2,known_for
3,movie_akas
4,movie_ratings
5,persons
6,principals
7,writers


##### Examination of `movie_basics` table

In [14]:
# Preview the tables of interest in the IMDB data
imdb_df = pd.read_sql("""
                      SELECT * 
                        FROM  movie_basics AS mb """, conn)

imdb_df.head()

,movie_id,primary_title,original_title,start_year,runtime_minutes,genres
0,tt0063540,Sunghursh,Sunghursh,2013,175.0,"Action,Crime,Drama"
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,2019,114.0,"Biography,Drama"
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,2018,122.0,Drama
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,2018,NaN,"Comedy,Drama"
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,2017,80.0,"Comedy,Drama,Fantasy"


In [15]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146144 entries, 0 to 146143
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   movie_id         146144 non-null  object 
 1   primary_title    146144 non-null  object 
 2   original_title   146123 non-null  object 
 3   start_year       146144 non-null  int64  
 4   runtime_minutes  114405 non-null  float64
 5   genres           140736 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 6.7+ MB


In [16]:
missing_values(imdb_df)

(Analysis of the movie_basics table data obtained from the IMDB dataset)

##### Examination of `persons` table

In [22]:
# Preview the tables of interest in the IMDB data
imdb_df = pd.read_sql("""
                      SELECT * 
                        FROM  persons
                      ;
                      """, conn)

imdb_df.head()

,person_id,primary_name,birth_year,death_year,primary_profession
0,nm0061671,Mary Ellen Bauder,NaN,NaN,"miscellaneous,production_manager,producer"
1,nm0061865,Joseph Bauer,NaN,NaN,"composer,music_department,sound_department"
2,nm0062070,Bruce Baum,NaN,NaN,"miscellaneous,actor,writer"
3,nm0062195,Axel Baumann,NaN,NaN,"camera_department,cinematographer,art_department"
4,nm0062798,Pete Baxter,NaN,NaN,"production_designer,art_department,set_decorator"


In [23]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 606648 entries, 0 to 606647
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   person_id           606648 non-null  object 
 1   primary_name        606648 non-null  object 
 2   birth_year          82736 non-null   float64
 3   death_year          6783 non-null    float64
 4   primary_profession  555308 non-null  object 
dtypes: float64(2), object(3)
memory usage: 23.1+ MB


In [24]:
missing_values(imdb_df)

(Analysis of the persons table data obtained from the IMDB dataset)

##### Examination of `movie_ratings` table

In [25]:
# Preview the tables of interest in the IMDB data
imdb_df = pd.read_sql("""
                      SELECT * 
                        FROM  movie_ratings
                      ;
                      """, conn)

imdb_df.head()

,movie_id,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [26]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73856 entries, 0 to 73855
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   movie_id       73856 non-null  object 
 1   averagerating  73856 non-null  float64
 2   numvotes       73856 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 1.7+ MB


(Analysis of the movie_ratings table data obtained from the IMDB dataset)

##### Examination of `movie_akas` table

In [27]:
imdb_df = pd.read_sql("""
                      SELECT * 
                        FROM  movie_akas
                      ;
                      """, conn)

imdb_df.head()

,movie_id,ordering,title,region,language,types,attributes,is_original_title
0,tt0369610,10,Джурасик свят,BG,bg,None,None,0.0
1,tt0369610,11,Jurashikku warudo,JP,None,imdbDisplay,None,0.0
2,tt0369610,12,Jurassic World: O Mundo dos Dinossauros,BR,None,imdbDisplay,None,0.0
3,tt0369610,13,O Mundo dos Dinossauros,BR,None,None,short title,0.0
4,tt0369610,14,Jurassic World,FR,None,imdbDisplay,None,0.0


In [28]:
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 331703 entries, 0 to 331702
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   movie_id           331703 non-null  object 
 1   ordering           331703 non-null  int64  
 2   title              331703 non-null  object 
 3   region             278410 non-null  object 
 4   language           41715 non-null   object 
 5   types              168447 non-null  object 
 6   attributes         14925 non-null   object 
 7   is_original_title  331678 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 20.2+ MB


In [43]:
missing_values(imdb_df)

(Analysis of the movie_akas table data obtained from the IMDB dataset)